<a href="https://colab.research.google.com/github/plthiyagu/AI-Engineering/blob/master/07-Machine%20Learning/Deploy_ML_Models_as_Service_with_FastAPI_%26_ColabCode_from_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colabcode
!pip install fastapi

     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.1-cp36-none-any.whl size=18822 sha256=851e1069f8828b3fb8e202a4ef03344c01b23422b9507a7718fe013ecab8e11d
  Stored in directory: /root/.cache/pip/wheels/94/01/05/d39efb8f6b40a411354b4168ca9dda99e6f8d586e458e97551
Successfully built pyngrok
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
  Found existing installation: nest-asyncio 1.5.1
    Uninstalling nest-asyncio-1.5.1:
      Successfully uninstalled nest-asyncio-1.5.1
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 
     |████████████████████████████████| 9.2MB 7.3MB/s 


In [1]:
from colabcode import ColabCode
from fastapi import FastAPI

In [2]:
cc = ColabCode(port=12000, code=False)

In [3]:
app = FastAPI()

@app.get("/")
async def read_root():
  return {"message": "Subscribe to @1littlecoder"}

In [ ]:
cc.run_app(app=app)

INFO:     Started server process [139]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://2cb1348ee058.ngrok.io" -> "http://localhost:12000"
INFO:     2603:6080:4306:eeb4:9902:9fb1:1282:d834:0 - "GET / HTTP/1.1" 200 OK
INFO:     2603:6080:4306:eeb4:9902:9fb1:1282:d834:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


In [5]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pickle

iris = load_iris()
model = GaussianNB()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1)
model_f = model.fit(X_train, y_train)

print("Model score: ", model.score(X_train, y_train))
print("Test Accuracy: ", model.score(X_test, y_test))

pickle.dump(model_f, open("model_gb.pkl", "wb"))

Model score:  0.9555555555555556
Test Accuracy:  1.0


In [6]:
%%writefile models.py
from pydantic import BaseModel, conlist
from typing import List


class Iris(BaseModel):
    data: List[conlist(float, min_items=4, max_items=4)]

Writing models.py


In [7]:
import pickle
import logging
from fastapi import FastAPI
from models import Iris

app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

# # Initialize logging
# my_logger = logging.getLogger()
# my_logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG, filename='logs.log')

model = None

@app.on_event("startup")
def load_model():
    global model
    model = pickle.load(open("model_gb.pkl", "rb"))

@app.post("/api", tags=["prediction"])
async def get_predictions(iris: Iris):
    try:
        data = dict(iris)['data']
        print(data)
        iris_types = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        prediction = list(map(lambda x: iris_types[x], model.predict(data).tolist()))
        log_proba = model.predict_log_proba(data).tolist()
        return {"prediction": prediction, "log_proba": log_proba}
    except:
        my_logger.error("Something went wrong!")
        return {"prediction": "error"}

In [8]:
cc.run_app(app=app)

INFO:     Started server process [61]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://4cec813da41a.ngrok.io" -> "http://localhost:12000"
INFO:     2603:6080:4306:eeb4:9902:9fb1:1282:d834:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2603:6080:4306:eeb4:9902:9fb1:1282:d834:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2603:6080:4306:eeb4:9902:9fb1:1282:d834:0 - "GET / HTTP/1.1" 404 Not Found
